In [10]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Load the data
loader = TextLoader('../data/10 Academy Cohort B - Weekly Challenge_ Week - 7.txt')
documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1757, which is longer than the specified 500
Created a chunk of size 1320, which is longer than the specified 500
Created a chunk of size 859, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500
Created a chunk of size 1345, which is longer than the specified 500
Created a chunk of size 2420, which is longer than the specified 500
Created a chunk of size 851, which is longer than the specified 500
Created a chunk of size 1221, which is longer than the specified 500
Created a chunk of size 852, which is longer than the specified 500


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

# Setup vector database
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

# Populate vector database
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()

In [19]:
pip install openai



Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define LLM
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define prompt template
template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use two sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | model
    | StrOutputParser() 
)

ImportError: Could not import openai python package. Please install it with `pip install openai`.

In [ ]:
from datasets import Dataset

questions = ["What is the business objective PromptlyTech", 
             "What are the key services provided by PromptlyTech?",
             "What is prompt engineering?",
            ]
ground_truths = [["PromptlyTech aims to revolutionize how businesses interact with LLMs, making the technology more accessible, efficient, and effective."],
                ["PromptlyTech focuses on Automatic Prompt Generation, Automatic Evaluation Data Generation, and Prompt Testing and Ranking services."],
                ["Prompt engineering is the craft of designing queries or statements to guide LLMs to produce desired outcomes."]]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions, 
    "answer": answers, 
    "contexts": contexts, 
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)